# Advanced Macro: Bewley-Aiyagari models
<br>
<div style="text-align: center;">
July 2020, Takeki Sunakawa, revised June 2021 Takeki Sunakawa
<br>
<div style="text-align: center;">    
Hitotsubashi University

# Introduction

- We extend our previous analysis in the two-person/two-period/two-state model.

- We will find that the households are ex ante indentical but ex post different due to incomplete markets.

- This type of model is known as a heterogeneous agent model.

- Heterogeneous agent models have implications on both micro and macro data.

## Saving problem revisited

- An individual chooses a sequence of consumption $\{c_{t}\}_{t=0}^{\infty}$ to maximize

$$
    E_{0} \sum_{t=0}^{\infty} \beta^{t} u(c_{t}).
$$

where $\beta\in(0,1)$ is discount factor and $u(c)$ satisfies the standard assumptions.

subject to

$$
    c_{t} + q b_{t+1} = b_{t} + e_{t}
$$

where

- $q_{t}$ is the price of risk-free bond which returns 1 unit of resources in the next period, 

- $b_{t+1}$ is the amount of risk-free bond purchased in period $t$, and

- $e_{t}$ is employment status which follows a two-state Markov chain.

- The borrowing constraint is given by $b_{t+1}\geq-\phi$.

## Markov chain

- $e_{t}$ takes either of two values $\{e_{1},e_{2}\}$.

- When $e_{t}=e_{1}$, the conditional probability of $e_{t+1}=e_{1}$, $\text{Prob}(e_{t+1}=e_{1}|e_{t}=e_{1})$, is given by $p_{11}$. 

- Similarly, when $e_{t}=e_{2}$, the conditional probability of $e_{t+1}=e_{2}$, $\text{Prob}(e_{t+1}=e_{2}|e_{t}=e_{2})$,  is given by $p_{22}$.

- The transition probability matrix is given by

$$
 P = \left[\begin{array}{cc}
 p_{11} & 1-p_{11} \\
 1-p_{22} & p_{22}
\end{array}\right].
$$

- Let $\pi_{t}$ be a $(2\times 1)$ vector of the distribution of $e_{t}$, $\pi_{t}(i)=\text{Prob}(e_{t}=e_{i})$, for $i=1,2$ in period $t$.

- Then, given $\pi_{0}$, we have

$$
\begin{align}
\pi_{1}' &= \pi_{0}'P, \\
 &= \left[\begin{array}{cc} \pi_{0}(1) & \pi_{0}(2) \end{array}\right] \left[\begin{array}{cc}
 p_{11} & 1-p_{11} \\
 1-p_{22} & p_{22}
\end{array}\right], \\
 &= \left[\begin{array}{cc} \pi_{0}(1)p_{11} + \pi_{0}(2)(1-p_{22}) & \pi_{0}(1)(1-p_{11}) + \pi_{0}(2)p_{22} \end{array}\right], \\
 &= \left[\begin{array}{cc} \pi_{1}(1) & \pi_{1}(2) \end{array}\right].
\end{align}
$$
<!--
$$
\underbrace{\pi_{1}'}_{(1\times2)} = \underbrace{\pi_{0}'}_{(1\times2)}\underbrace{P}_{(2\times2)}.
$$
-->

- Also,

$$
\begin{align}
\pi_{2}' &= \pi_{0}'P^{2}, \\
\pi_{3}' &= \pi_{0}'P^{3}, \\
& \vdots \\
\pi_{t}' &= \pi_{0}'P^{t}.
\end{align}
$$

- We can obtain the time-invariant distribution as $\pi=\lim_{t \rightarrow \infty} \pi_{0}'P^{t}$.

- Or, $\pi=P'\pi$ implies

$$
  (I-P')\pi = 0
$$

Then $\pi$ is an eigenvector of the matrix $P′$ associated with unit eigenvalue.

In [1]:
P = zeros(2,2)
P[1,1] = .7
P[1,2] = .3
P[2,1] = .2
P[2,2] = .8

# computing the invariant distribution
# iterative method
p0 = [.5,.5]

for t in 1:10000
    p0 = P'*p0
end
println(p0)

# eigenvalue decomposition
using LinearAlgebra

F = eigen(P')
println(F.values) # the second element of Lambda = 1
println(F.vectors[:,2]/sum(F.vectors[:,2]))

[0.3999999999999998, 0.5999999999999998]
[0.5, 1.0]
[0.39999999999999997, 0.6]


## Dynamic programming

- We write the Bellman equation as

$$
\begin{align}
    V^{*}(b,e_{1}) = \max_{b'\geq-\phi}\left\{ \ln(b+e_{1}-qb')+\beta\left(p_{11}V_{1}(b')+p_{12}V_{2}(b')\right)\right\}, \\
    V^{*}(b,e_{2}) = \max_{b'\geq-\phi}\left\{ \ln(b+e_{2}-qb')+\beta\left(p_{21}V_{1}(b')+p_{22}V_{2}(b')\right)\right\},
\end{align}
$$

taking the function $V_{k}(b) \equiv V(b,e_{k})$ for $k=1,2$ as given.

- We update the value function iteratively until the function converges, i.e., $\left\Vert V_{k}^{*}(b)-V_{k}(b)\right\Vert \leq\epsilon$ for each $k=1,2$.

- We need to approximate $V_{k}(b')$, and numerically solve the maximization problem with regard to $b'$.

## Successive approximation

- Let $b_{i}\in\mathcal{B}=\{b_{1},b_{2},...,b_{n_{b}}\}$ where $b_{1}<b_{2}<...<b_{n_{b}}$. These are called grid points.

- $V_{k}(b)$ is approximated by a $(n \times 1)$ vector of the values $\{V_{k}(b_{i})\}_{i=1}^{n}$ for each $k=1,2$.


<!--- $V(b)=\{V_{k}(b)\}_{k=0}^{1}$ is a $(2 \times n_{b})$ vector.-->

- We can successively update the approximation as follows:

0. Construct a matrix for the current utility $\underset{(2 \times n \times n)}{\mathbf{U}}=\{\{u_{kij}\}\}$

  - <span style="color:blue">For each index for the employment status $k=1,2$:
  
  - Let $i$ be the index for the current period's asset. That is, $b_{i}=b$.
  
  - Also let $j$ be the index for the next period's aseet. That is, $b_{j}=b'$.

  - The utility matrix is given by
  
  $$
U_k = \left[
\begin{array}{ccccc}
u_{k11} & \cdots & u_{k1j} & \cdots & u_{k1n}\\
\vdots & \ddots &        &        & \vdots \\
u_{ki1} &        & u_{kij} &        & u_{kin} \\
\vdots &        &        & \ddots & \vdots \\
u_{kn1} & \cdots & u_{knj} & \cdots & u_{knn}
\end{array}
\right]
  $$    
  
  for $k=1,2$.
    
  - Then, if $c_{kij}=b_{i}+e_{k}-qb_{j}>0$, set
  $$
    u_{kij} = \ln c_{kij}
  $$
  Otherwise, set a large negative number (say -10000000000) to $u_{kij}$.

1. For each pair of indices of the current period $i=1,2,...,n$ and $k=1,2$ and given a vector of the values $\{V_{k}(b_{j})\}_{j=1}^{n}$, 

<!--   - Let $\underset{(n \times 1)}{\mathbf{u}}=[u_{ki0},...,u_{kij},...,u_{kin-1}]$ and $\underset{(n \times 1)}{\mathbf{v}_{k}}=[V_{k}(b_{0}),...,V_{k}(b_{j}),...V_{k}(b_{n-1})]$. -->

  - <span style="color:blue">Compute the expected value function
$$
    \left[
    \begin{array}{c}
    E_{k}V(b_{1})\\
    \vdots\\
    E_{k}V(b_{j})\\
    \vdots\\
    E_{k}V(b_{n})
    \end{array}
    \right] = p_{k1}
    \left[
    \begin{array}{c}
    V_{1}(b_{1})\\
    \vdots\\
    V_{1}(b_{j})\\
    \vdots\\
    V_{1}(b_{n})
    \end{array}
    \right] + p_{k2}
    \left[
    \begin{array}{c}
    V_{2}(b_{1})\\
    \vdots\\
    V_{2}(b_{j})\\
    \vdots\\
    V_{2}(b_{n})
    \end{array}
    \right]
$$
    
<!-- $$
    \underset{(n \times 1)}{E_{k}\mathbf{v}} = p_{k0}\mathbf{v}_{0} + p_{k1}\mathbf{v}_{1}
$$ -->
    
  - Find the index of the next period $j$ by
$$
    V^{*}_{k}(b_{i})=\max_{j}\left\{
    \left[
    \begin{array}{c}
    u_{ki1}\\
    \vdots\\
    u_{kij}\\
    \vdots\\
    u_{kin}
    \end{array}
    \right]+
    \beta
    \left[
    \begin{array}{c}
    E_{k}V(b_{1})\\
    \vdots\\
    E_{k}V(b_{j})\\
    \vdots\\
    E_{k}V(b_{n})
    \end{array}
    \right]
    \right\}
$$

<!-- $$
    V^{*}_{k}(b_{i})=\max_{j}\left\{ \mathbf{u} + \beta E_{k}\mathbf{v}\right\}
$$ -->
    
  - Then we obtain a vector of the values $\{V^{*}_{k}(b_{i})\}_{i=1}^{n}$ for each $k=1,2$, which approximate the (new) value function.

2. Update the old value function $\{V_{k}(b_{i})\}_{i=1}^{n}$ with the new value function $\{V^{*}_{k}(b_{i})\}_{i=1}^{n}$. Iterate 1-2
until convergence.

- $q = \beta$: $g(b,e_{1})$ is always larger than $b$, whereas $g(b,e_{2})$ is always smaller than $b$. 
<br>
<div align="center">
<img src="./bpol.png",width="1200",height="400">
</div>

- $q = 1.025$: $b'=g(b,e_{1})$ crosses the 45 degree line from above around at $b'=b=0$.
<br>
<div align="center">
<img src="./bpol2.png",width="1200",height="400">
</div>

# Huggett model

- Mark Huggett (1993) studies an infinite-period economy with uncertainty, which is a simple extension to the previous two-person/two-period/two-state economy.

- There is a continuum of households $i\in[0,1]$. 

- Each of households has access to a single type of asset which it can borrow or lend at a rate $q$. The households cannot borrow exceeding the limit $\phi$.

- The households are ex ante indentical but ex post different due to incomplete markets.

## Wealth-employment distributions

- By solving the household's saving problem, we have

$$
    b' = g(b,e_{k})
$$

for $k=1,2$.

- Define

$$
    \lambda_{t}(b,e_{k}) = \text{Prob}(b_{t}=b,e_{t}=e_{k})
$$

- The transition matrix $P$ and the policy function $b'=g(b,e)$ induce

$$
\begin{align}
    \lambda_{t+1}(b',e_{l}) =& \sum_{b_{t}}\sum_{e_{t}}\text{Prob}(b_{t+1}=b'|b_{t}=b,e_{t}=e_{k}) \\
    &\cdot\underbrace{\text{Prob}(e_{t+1}=e_{l}|e_{t}=e_{k})}_{p_{kl}}\cdot\underbrace{\text{Prob}(b_{t}=b,e_{t}=e_{k})}_{\lambda_{t}(b,e_{k})}, \\
    =& \sum_{b_{t}}\sum_{e_{t}}\text{Prob}(b_{t+1}=b'|b_{t}=b,e_{t}=e_{k})\cdot{p_{kl}}\cdot\lambda_{t}(b,e_{k}).
\end{align}
$$

- The time invariant distribution is given by $\lambda(b,e)$.

## Closing the model

- The asset markets clear. That is,

$$
    \sum_{b,e} b\lambda(b,e) = 0,
$$

which determines the equilibrium price $q>\beta$. Note that $\sum_{b,e} \lambda(b,e) = 1$.

## Stationary equilibrium

Definition: Given a borrowing limit $\phi$, a stationary equilibrium is a debt price $q$, a policy function $g(b,e)$, and a stationary distribution $\lambda(b,e)$ for which

(a) Given $q$, the policy function solves the household's optimum problem;

(b) The probability distribution $\lambda(b,e)$ is the invariant distribution of the Markov chain on $(b,e)$ induced by the Markov chain on $e$ and the optimal policy $g(b,e)$;

(c) The asset market clears
$$
    \sum_{b,e} b\lambda(b,e) = 0,
$$

## Calculating wealth-employment distributions

- How do we calculate $\lambda(b,e)$ numerically? <!--$\text{Prob}(b_{t+1}=b'|b_{t}=b,e_{t}=e_{k})$?-->

- Let $b_{i}\in\mathcal{B}$ and $b_{j}\in\mathcal{B}$ where $\mathcal{B}=\{b_{1},b_{2},...,b_{n_{b}}\}$ and $b_{1}<b_{2}<...<b_{n_{b}}$.

- Then, if $b_{j}=g(b_{i},e_{k})$,

$$
\text{Prob}(b_{t+1}=b_{j}|b_{t}=b_{i},e_{t}=e_{k}) = 1.
$$

- If $b'\in[b_{j},b_{j+1}]$,

$$
\begin{align}
\text{Prob}(b_{t+1}=b_{j}|b_{t}=b_{i},e_{t}=e_{k}) &= w, \\
\text{Prob}(b_{t+1}=b_{j+1}|b_{t}=b_{i},e_{t}=e_{k}) &= 1-w,
\end{align}
$$

where $w=\frac{b_{j+1}-b'}{b_{j+1}-b_{j}}$.

[graph]

- Let $(i(m),k(m))$ be a pair of functions of an index for the current period $m=1,...,n_{b},n_{b}+1,...,2n_{b}$.

- For example, $n_{b}=3$,

$$
\begin{align}
  (i(1),k(1)) &= (1,1), \\
  (i(2),k(2)) &= (2,1), \\
  (i(3),k(3)) &= (3,1), \\
  (i(4),k(4)) &= (1,2), \\
  (i(5),k(5)) &= (2,2), \\
  (i(6),k(6)) &= (3,2).
\end{align}
$$

- Similarly, let $(j(n),l(n))$ be a pair of functions of an index for the next period $n=1,...,n_{b},n_{b}+1,...,2n_{b}$.

- Then the distribution of assets and employment status can be stacked into a vector

$$
  \mathbf{\lambda}_{t} = \left[\begin{array}{ccccc}\lambda_{t,1}, & \cdots, & \lambda_{t,m}, & \cdots, & \lambda_{t,N}\end{array}\right]'
$$

where $\lambda_{t,m}=\lambda_{t}(b_{i(m)},e_{k(m)})$. Note that $N=2n_{b}$. 

- Similarly,

$$
  \mathbf{\lambda}_{t+1} = \left[\begin{array}{ccccc}\lambda_{t+1,1}, & \cdots, & \lambda_{t+1,n}, & \cdots, & \lambda_{t+1,N}\end{array}\right]'
$$

where $\lambda_{t+1,n}=\lambda_{t}(b_{j(n)},e_{l(n)})$.

- Then we have the transition matrix for $\lambda_{t}$

$$
\underbrace{\left[\begin{array}{c}
\lambda_{t+1,1} \\
\vdots \\
\lambda_{t+1,n} \\
\vdots \\
\lambda_{t+1,N}\end{array}\right]}_{\underset{(N \times 1)}{\lambda_{t+1}}}
= 
\underbrace{\left[\begin{array}{ccccc}
a_{11} & \cdots & a_{1m} & \cdots & a_{0N} \\
\vdots & & \vdots & & \vdots \\
a_{n1} & \cdots & a_{nm} & \cdots & a_{nN} \\
\vdots & & \vdots & & \vdots \\
a_{N1} & \cdots & a_{Nm} & \cdots & a_{NN} \end{array}\right]}_{\underset{(N \times N)}{A'}}
\underbrace{\left[\begin{array}{c}
\lambda_{t,1} \\
\vdots \\
\lambda_{t,m} \\
\vdots \\
\lambda_{t,N}\end{array}\right]}_{\underset{(N \times 1)}{\lambda_{t}}}
$$

where

$$
\begin{align}
    a_{nm} &= \text{Prob}(b_{t+1}=b_{j(n)}|b_{t}=b_{i(m)},e_{t}=e_{k(m)})\cdot\text{Prob}(e_{t+1}=e_{l(n)}|e_{t}=e_{k(m)}), \\
    &= p_{kl}
\end{align}
$$

- Note that this is a sparse matrix. Most of the elements are zero.

In [ ]:
# transition matrix
function transition_matrix(gmat0,Pe)

    Ne = size(gmat0,1)
    Nb = size(gmat0,2)
    A  = zeros(Ne*Nb,Ne*Nb)

    for k in 1:Ne         # index for today's employment

        for i in 1:Nb     # index for tomorrow's asset

            j = Int(gmat0[k,i]) # index for tomorrow's asset, from the policy function

            for l in 1:Ne # index for tomorrow's employment

                m = Nb*(k-1)+i # m = 1,...,2Nb
                n = Nb*(l-1)+j # n = 1,...,2Nb
                A[m,n] = Pe[k,l]
                
            end
            
        end
        
    end
                
    return A
    
end

- This is a Markov chain for $\lambda$.  The time-invariant distribution is obtained by

$$
  \lambda'=\lim_{t \rightarrow \infty} \lambda_{0}'A^{t}.
$$

- Or, eigenvalue and eigenvector decomposition can be used as $\lambda=A'\lambda$ holds.

In [ ]:
# distribution
function dist_iter(lambda0,A)

    N = size(lambda0,1)
    lambda1 = zeros(N)

    diff = 1e+4
    
    while diff>1e-14
        lambda1 = A'*lambda0
        diff    = maximum(abs.(lambda1-lambda0))
#         print(diff)
        lambda0 = lambda1/sum(lambda1)
    end

    return lambda0
    
end

- The aggregate demand for asset is

$$
  z = \sum_{b,e} b\lambda(b,e).
$$

In [ ]:
# the aggregate demand for asset
# NOTE: lambda is (2*nb times 1) vector, the first nb elements are for k=0 and the second nb elements are for k=1
z = sum(bgrid.*lambda0[1:nb]) + sum(bgrid.*lambda0[nb+1:2*nb])

## Computing the equilibrium price

1. Fix $q\geq\beta$ and solve the household problem for a policy function and an associated stationary distribution.


2. Check if the asset market clears. If $z = \sum b\lambda(b,e) > 0$, raise $q$. Otherwise, lower $q$.


3. Iterate 1-2 until we obtain $q$ at which excess demand for loans is zero.

- The lower is $q$, the higher is $z$. In theory, $z=\infty$ at $q=\beta$. 

## Aiyagari model

<!-- Each of households has access to a single type of asset which it can borrow or lend at a rate $q$. The households cannot borrow exceeding the limit $\phi$.-->

- Rao Aiyagari (1994) studies a version of the saving problem in which physical capital is the only asset.

- The households' supply and the firm's demand are met in the asset market.

## Household's problem

- For given $(r,w)$, household $i$ maximizes

$$
    E_{0} \sum_{t=0}^{\infty} \beta^{t} u(c_{t})
$$

subject to

$$
    c_{t} + a_{t+1} \leq (1+r) a_{t} + w e_{t}
$$

and the borrowing constraint $a_{t+1}\geq-b$.

$c_{t}$ is consumption, $a_{t}$ is asset holding, $e_{t}$ is employment status.

- $e_{t}\in\{e_{1},\cdots,e_{n_{e}}\}$ follows a Markov chain with the transition matrix $P$.

- The time invariant distribution is $\lambda(a,e)$. The aggregate supply for capital is

$$
  K = \sum \int a \lambda(a,e) da.
$$

- The aggregate employment is

$$
  N = \pi'\mathbf{e},
$$

where $\pi$ is the invariant distribution associated with $P$ and $\mathbf{e}=[e_{1},\cdots,e_{n_{e}}]'$.

## Firm

- There is an aggregate production function, which determines

$$
\begin{align}
    r &= \partial F(K,N)/\partial N - \delta, \\
    w &= \partial F(K,N)/\partial K,
\end{align}
$$

where $F(K,N) = AK^{\alpha}N^{1-\alpha}$ and $\alpha\in(0,1)$.

## Stationary equilibrium

Definition: A stationary equilibrium is a policy function $g(a,e)$, a stationary distribution $\lambda(a,e)$, and positive real numbers $(K,r,w)$, for which

(a) Given $(r,w)$, the policy function solves the household's optimum problem;

(b) The probability distribution $\lambda(a,e)$ is the invariant distribution of the Markov chain on $(a,e)$ induced by the Markov chain on $e$ and the optimal policy $g(a,e)$;

(c) The asset market clears
$$
    K = \sum_{a,e} \lambda(a,e)g(a,e).
$$

(d) The prices $(r,w)$ satisfy
$$
\begin{align}
    r &= \partial F(K,N)/\partial N - \delta, \\
    w &= \partial F(K,N)/\partial K.
\end{align}
$$

- Let $\lambda=\beta^{-1}-1$. $Ea(r)$ is the aggregate supply for capital and $K(r)$ is the aggregate demand for capital.
  - The demand exceeds the supply at $r=r_{1}$ and the supply exceeds the demand at $r=r_{2}$.
  - The supply is infinite at $r=\lambda$.
  - The demand and the supply are met at $r^{*}<\lambda$, which implies $\beta(1+r^{*})<1$.
<br>
<div align="center">
<img src="./aiyagari_fig.png",width="600",height="400">
</div>